# Working Section

In [21]:
# Connect / authenticate to Jira
# Retrieve a list of the Jira Tickets on the Roadmap Kanban board

# Connect / authenticate to Google Slides
# Create new placeholder slides for each quarter (where the roadmaps will be populated)

# Iterate through the list of Jira tickets
    # Select the slide to place the ticket on based on the "Product - X" component
    # Select the location on the slide based on the "Status" (e.g., Q1)
    # Create the request body for the initiative and add to growing list

# Write back all of the slide updates in one request to Google Slides

## General Purpose Stuff

In [17]:
from dataclasses import dataclass
import uuid
import json
from dataclasses import dataclass
from datetime import datetime
from jira import JIRA
from getpass import getpass

## Google Slides

### Authenticate

In [4]:
from google_auth_oauthlib.flow import InstalledAppFlow, Flow
from googleapiclient.discovery import build


def google_slides_connect():
    '''Authenticate to Google Slides and generate a service to be leveraged when updating/populating slides

    Returns: 
        (googleapiclient.discovery.Resource)
    '''
    
    flow = InstalledAppFlow.from_client_secrets_file(
        'google-credentials.json',
        scopes=['https://www.googleapis.com/auth/presentations'])
    
    credentials = flow.run_local_server()
    
    return build('slides', 'v1', credentials=credentials)

In [3]:
slides_service = google_slides_connect()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1076520620839-3cadpb9secdetc938k51obvoc2tl79sc.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpresentations&state=gdvT0kJ8EbChyZhvaUFQJWA54wQF5t&access_type=offline


### Update Slides

In [5]:
def updateSlides(service, **kwargs):
    '''General function for pushing updates to a specific slide deck'''
    return service.presentations().batchUpdate(**kwargs).execute()

### Create Slide

In [6]:
def gen_header_slide_req(title):
    '''Creates a request body for a new slide

    Args: 
        title (Str): the title of the slide

    Returns: 
        (List(dict)): request body to generate roadmap item
        (str): Google Slides id of the object created
    '''

    titleId = str(uuid.uuid4())
    slideId = str(uuid.uuid4())

    request_body = [
        {
            'createSlide': {
                'objectId':slideId,
                'slideLayoutReference': {
                    'predefinedLayout': 'SECTION_HEADER'
                },
                'placeholderIdMappings': [
                    {
                        'objectId': titleId,
                        'layoutPlaceholder': {'type': 'TITLE', 'index': 0}
                    }
                ]
            }
        },
        {'insertText': {'objectId': titleId, 'text': title}},
    ]
    
    return request_body, slideId

In [34]:
def gen_roadmap_slide_req(
    title,
    roadmap_slide_config
):
    '''Creates a request body for a new slide

    Args: 
        title (Str): the title of the slide
        roadmap_slide_config (dict): configuration for the roadmap slides
        ...
        
    Returns: 
        (List(dict)): request body to generate roadmap item
        (str): Google Slides id of the object created
    '''

    left_header_config = roadmap_slide_config['left_header'] 
    left_main_config = roadmap_slide_config['left_main'] 
    timeline_arrow_config = roadmap_slide_config['timeline_arrow'] 
    quarter_marker_config = roadmap_slide_config['quarter_marker'] 
    quarters_config = roadmap_slide_config['quarters'] 
    roadmap_box_config = roadmap_slide_config['roadmap_box']
    columns_config = roadmap_slide_config['columns']
    
    title_id = str(uuid.uuid4())
    slide_id = str(uuid.uuid4())    
    left_header_element_id = str(uuid.uuid4())
    left_main_element_id = str(uuid.uuid4())

    left_main_locx=left_header_config["locx"]
    
    # timeline arrow
    timeline_arrow_start_locx = left_header_config["locx"] + left_header_config["width"]
    timeline_arrow_start_locy = left_header_config["locy"] + left_header_config["height"]/2
    timeline_arrow_element_id = str(uuid.uuid4())
    
    
    slide_req = [{
        'createSlide': {
            'objectId':slide_id,
            'slideLayoutReference': {
                'predefinedLayout': 'TITLE_AND_TWO_COLUMNS'
            },
            'placeholderIdMappings': [
                {
                    'objectId': title_id,
                    'layoutPlaceholder': {'type': 'TITLE', 'index': 0}
                }
            ]
        }
    }]

    title_req = [{'insertText': {'objectId': title_id, 'text': title}}]
    
    left_main_req = [{
            "createShape": {
                "objectId": left_main_element_id,
                "shapeType": left_main_config["shape_type"],
                "elementProperties": {
                    "pageObjectId": slide_id,
                    "size": {
                        "height": {"magnitude": left_main_config["height"], "unit": "PT"}, 
                        "width": {"magnitude": left_main_config["width"], "unit": "PT"}
                    },
                    "transform": {
                        "scaleX": 1,
                        "scaleY": 1,
                        "translateX": left_main_locx,
                        "translateY": left_main_config["locy"],
                        "unit": "PT",
                    }
                }
            }
        },
        {
            "updateShapeProperties" : {
                "fields" : "contentAlignment, \
                    outline.outlineFill.solidFill.alpha, \
                    shapeBackgroundFill.solidFill.color.themeColor",
                "objectId": left_main_element_id,
                "shapeProperties" : {
                    "contentAlignment": "TOP",
                    "outline": {"outlineFill": {"solidFill":{"alpha":0}}},
                    "shapeBackgroundFill":{"solidFill": {"color": {"themeColor":left_main_config["color"]}}}
                }
            }
        },
        {
            "insertText": {
                "objectId": left_main_element_id,
                "insertionIndex": 0,
                "text": f"\n\n\n{left_main_config['text']}"
            }
        },
        {
            "updateTextStyle" : {
                "fields": "fontFamily, fontSize.magnitude, fontSize.unit",
                "objectId": left_main_element_id,
                "style" : {
                    "fontFamily": "Manrope",
                    "fontSize": {"magnitude": left_main_config["font_size"], "unit":"PT"}
                },
            }
        },
        {
            "updateParagraphStyle" : {
                "fields": "alignment",
                "objectId": left_main_element_id,
                "style" : {
                    "alignment": "START",
                },
            }
        }]
    
    left_header_req = [{
            "createShape": {
                "objectId": left_header_element_id,
                "shapeType": left_header_config["shape_type"],
                "elementProperties": {
                    "pageObjectId": slide_id,
                    "size": {
                        "height": {"magnitude": left_header_config["height"], "unit": "PT"}, 
                        "width": {"magnitude": left_header_config["width"], "unit": "PT"}
                    },
                    "transform": {
                        "scaleX": 1,
                        "scaleY": 1,
                        "translateX": left_header_config["locx"],
                        "translateY": left_header_config["locy"],
                        "unit": "PT",
                    }
                }
            }
        },
        {
            "updateShapeProperties" : {
                "fields" : "contentAlignment, \
                    outline.outlineFill.solidFill.alpha, \
                    shapeBackgroundFill.solidFill.color.themeColor",
                "objectId": left_header_element_id,
                "shapeProperties" : {
                    "contentAlignment": "TOP",
                    "outline": {"outlineFill": {"solidFill":{"alpha":0}}},
                    "shapeBackgroundFill":{"solidFill": {"color": {"themeColor":left_header_config["color"]}}}
                }
            }
        },
        {
            "insertText": {
                "objectId": left_header_element_id,
                "insertionIndex": 0,
                "text": title
            }
        },
        {
            "updateTextStyle" : {
                "fields": "bold, fontFamily, fontSize.magnitude, fontSize.unit, \
                    foregroundColor.opaqueColor.themeColor",
                "objectId": left_header_element_id,
                "style" : {
                    "bold":True,
                    "fontFamily": "Manrope",
                    "fontSize": {"magnitude": left_header_config["font_size"], "unit":"PT"},
                    "foregroundColor": {"opaqueColor": {"themeColor":"LIGHT1"}}
                },
            }
        },
        {
            "updateParagraphStyle" : {
                "fields": "alignment",
                "objectId": left_header_element_id,
                "style" : {
                    "alignment": "CENTER",
                },
            }
        }]

    timeline_arrow_req = [{
            "createLine": {
                "objectId": timeline_arrow_element_id,
                "lineCategory": 'STRAIGHT',
                "elementProperties": {
                    "pageObjectId": slide_id,
                    "size": {
                        "height": {"magnitude": 1, "unit": "PT"}, 
                        "width": {"magnitude": timeline_arrow_config["width"], "unit": "PT"}
                    },
                    "transform": {
                        "scaleX": 1,
                        "scaleY": 1,
                        "translateX": timeline_arrow_start_locx,
                        "translateY": timeline_arrow_start_locy,
                        "unit": "PT",
                    }
                }
            }
        },
        {
            "updateLineProperties" : {
                "fields" : "endArrow, lineFill.solidFill.color.themeColor, weight",
                "objectId": timeline_arrow_element_id,
                "lineProperties" : {
                    "endArrow": "FILL_ARROW",
                    "lineFill":{"solidFill": {"color": {"themeColor":timeline_arrow_config["color"]}}},
                    "weight": {"magnitude": timeline_arrow_config["weight"], "unit": "PT"}
                }
            }
        }]

    quarter_marker_reqs = []

    quarter_width = (timeline_arrow_config["width"] - roadmap_box_config['x_padding']*2) / len(columns_config)
    quarter_locx = timeline_arrow_start_locx + roadmap_box_config['x_padding']

    text_box_width = 200
    
    for col_num, column in enumerate(columns_config):
        
        quarter_marker_element_id = str(uuid.uuid4())
        quarter_textbox_element_id = str(uuid.uuid4())
    
        quarter_marker_reqs += [
            {
                "createShape": {
                    "objectId": quarter_marker_element_id,
                    "shapeType": quarter_marker_config["shape_type"],
                    "elementProperties": {
                        "pageObjectId": slide_id,
                        "size": {
                            "height": {"magnitude": quarter_marker_config["height"], "unit": "PT"}, 
                            "width": {"magnitude": quarter_marker_config["width"], "unit": "PT"}
                        },
                        "transform": {
                            "scaleX": 1,
                            "scaleY": 1,
                            "translateX": quarter_locx + quarter_width/2 - quarter_marker_config['width']/2 + (quarter_width * col_num),
                            "translateY": timeline_arrow_start_locy - quarter_marker_config["height"]/2,
                            "unit": "PT",
                        }
                    }
                }
            },
            {
                "updateShapeProperties" : {
                    "fields" : "outline.outlineFill.solidFill.alpha, \
                        shapeBackgroundFill.solidFill.color.themeColor",
                    "objectId": quarter_marker_element_id,
                    "shapeProperties" : {
                        "outline": {"outlineFill": {"solidFill":{"alpha":0}}},
                        "shapeBackgroundFill":{"solidFill": {"color": {"themeColor":quarter_marker_config["color"]}}}
                    }
                }
            },
            {
                "createShape": {
                    "objectId": quarter_textbox_element_id,
                    "shapeType": "RECTANGLE",
                    "elementProperties": {
                        "pageObjectId": slide_id,
                        "size": {
                            "height": {"magnitude": quarter_marker_config["height"], "unit": "PT"}, 
                            "width": {"magnitude": text_box_width, "unit": "PT"}
                        },
                        "transform": {
                            "scaleX": 1,
                            "scaleY": 1,
                            "translateX": quarter_locx + quarter_width/2 - text_box_width/2 + (quarter_width * col_num),
                            "translateY": timeline_arrow_start_locy - quarter_marker_config["height"]*1.75,
                            "unit": "PT",
                        }
                    }
                }
            },
            {
                "updateShapeProperties" : {
                    "fields" : "outline.outlineFill.solidFill.alpha, \
                        shapeBackgroundFill.solidFill.alpha",
                    "objectId": quarter_textbox_element_id,
                    "shapeProperties" : {
                        "outline": {"outlineFill": {"solidFill":{"alpha":0}}},
                        "shapeBackgroundFill":{"solidFill":{"alpha":0}}
                    }
                }
            },
            {
                "insertText": {
                    "objectId": quarter_textbox_element_id,
                    "insertionIndex": 0,
                    "text": column['label']
                }
            },
            {
                "updateTextStyle" : {
                    "fields": "bold, fontFamily, fontSize.magnitude, foregroundColor.opaqueColor.themeColor",
                    "objectId": quarter_textbox_element_id,
                    "style" : {
                        "bold": True,
                        "fontFamily": quarter_marker_config['font'],
                        "fontSize": {"magnitude": quarter_marker_config["font_size"], "unit":"PT"},
                        "foregroundColor": {"opaqueColor": {"themeColor":quarter_marker_config['font_color']}}
                    }
                }
            }                
        ]
    
    request_body = slide_req + title_req + left_main_req + left_header_req + timeline_arrow_req + quarter_marker_reqs
    
    return request_body, slide_id

In [112]:
new_slide_req, slide_id = gen_slide_req("My Awesome Slide", slide_type='roadmap')

res = updateSlides(
    service = slides_service,
    presentationId = '1XDRX_Bq39GTT8_6r2oorklhkw-MuIpn-VqFUSHn_Fdo',
    body = {'requests': new_slide_req}
)

### Roadmap Slide Item

In [11]:
def gen_roadmap_item_req(page_id, width, locx, locy, tagline, description, link, roadmap_box_config):
    '''Creates the necessary request body for generating a roadmap item in Google Slides

    Args: 
        page_id (str): id of the Google Slide page
        width (int): pixel width of roadmap box
        locx (int): location on x-axis
        loxy (int): location on y-axis
        tagline (str): short tagline for the roadmap item
        description (str): 1 - 2 sentences describing the roadmap intiative
        link (str): URL to the jira roadmap item
        roadmap_box_config (dict): configuration of the roadmap boxes

    Returns: 
        (List(dict)): request body to generate roadmap item
        (str): Google Slides id of the object created
    '''
    
    # Properties for all roadmap item boxes
    SHAPE_TYPE='ROUND_RECTANGLE'
    
    ptHeight = {"magnitude": roadmap_box_config["height"], "unit": "PT"}
    ptWidth = {"magnitude": width, "unit": "PT"}

    element_id = str(uuid.uuid4())
    
    request_body = [
        {
            "createShape": {
                "objectId": element_id,
                "shapeType": roadmap_box_config["shape_type"],
                "elementProperties": {
                    "pageObjectId": page_id,
                    "size": {"height": ptHeight, "width": ptWidth},
                    "transform": {
                        "scaleX": 1,
                        "scaleY": 1,
                        "translateX": locx,
                        "translateY": locy,
                        "unit": "PT",
                    }
                }
            }
        },
        {
            "updateShapeProperties" : {
                "fields" : "contentAlignment, \
                    outline.outlineFill.solidFill.color.themeColor, \
                    link.url, \
                    shapeBackgroundFill.solidFill.color.themeColor",
                "objectId": element_id,
                "shapeProperties" : {
                    "contentAlignment": "TOP",
                    "outline": {"outlineFill": {"solidFill": {"color": {"themeColor":roadmap_box_config["outline_color"]}}}},
                    "link": {"url": link},
                    "shapeBackgroundFill":{"solidFill": {"color": {"themeColor":roadmap_box_config["fill_color"]}}}
                }
            }
        },
        {
            "insertText": {
                "objectId": element_id,
                "insertionIndex": 0,
                "text": tagline + '\n' + description,
            }
        },
        {
            "updateTextStyle" : {
                "fields": "bold, fontFamily, fontSize.magnitude, fontSize.unit",
                "objectId": element_id,
                "style" : {
                    "fontFamily": "Manrope",
                    "fontSize": {"magnitude": roadmap_box_config["font_size"], "unit":"PT"}
                },
            }
        },
        {
            "updateParagraphStyle" : {
                "fields": "alignment",
                "objectId": element_id,
                "style" : {
                    "alignment": "START",
                },
            }
        },
        {
            "updateTextStyle" : {
                "fields": "bold",
                "objectId": element_id,
                "style" : {
                    "bold": True,
                },
                "textRange": {"endIndex": len(tagline), "startIndex": 0, "type": "FIXED_RANGE"}
            }
        }
    ]

    return request_body, element_id

In [113]:
new_shape_req, shape_id = gen_roadmap_item_req(
    page_id=slide_id,
    tagline='Roadmap Initiative 1',
    description='Some sort of description representing the initiative. A bit longer than the tagline.',
    locx=10,
    locy=100,
    link='https://www.google.com'
)

TypeError: gen_roadmap_item_req() missing 3 required positional arguments: 'height', 'width', and 'font_size'

In [9]:
res = updateSlides(
    service = slides_service,
    presentationId = '1XDRX_Bq39GTT8_6r2oorklhkw-MuIpn-VqFUSHn_Fdo',
    body = {'requests': new_shape_req}
)

## Jira

### Connect / Authenticate

In [7]:
from jira import JIRA
from getpass import getpass

jira_cloud = JIRA(basic_auth=("hunter.beck@cognite.com", getpass('Jira API Key:')), options={'server': 'https://cognitedata.atlassian.net/'})

Jira API Key: ········


### Get Roadmap Jira Issues

In [6]:
@dataclass
class JiraRoadmapIssue:
    summary:str
    description:str
    jira_id:str
    product_categories:list
    quarter:int
    jira_link:str

def get_roadmap_issues(jira_service, jql_filter, product_category_mode, product_category_prefix):
    '''Retrieves all of the Roadmap Initiatives in Jira Cloud

    Args: 
        jira_service (JIRA): authenticated service used for interacting with jira
        jql_filter (str): JQL compliant string to filter the specific issues
        product_category_mode (str): 'components' or 'labels' 
    '''
    
    roadmap_issue_ids = jira_service.search_issues(
        jql_str=jql_filter,
        maxResults=None
    )

    if roadmap_issue_ids:
        
        roadmap_issues = []
        
        for issue_id in roadmap_issue_ids: 

            issue = jira_service.issue(id=issue_id)

            if product_category_mode == 'components':

                issue_categories = [comp.name for comp in issue.fields.components]
            
            elif product_category_mode == 'labels':
                
                issue_categories = issue.fields.labels

            else:
                raise Exception(f"product_category_mode: '{product_category_mode}' is not valid. Choose from either 'components' or 'labels'")

            filtered_categories = []

            for category in issue_categories:

                if category.startswith(product_category_prefix):

                    filtered_categories.append(category[len(product_category_prefix):])
            
            roadmap_issues.append(JiraRoadmapIssue(
                summary=issue.fields.summary,
                description=issue.fields.description,
                jira_id=issue_id.id,
                product_categories=filtered_categories,
                quarter=int(issue.fields.status.name[1:]),
                jira_link=issue.permalink()
            ))
            
        return roadmap_issues

    else: 

        raise Exception("No jira issues were found with the provided JQL Filter")

NameError: name 'dataclass' is not defined

In [4]:
from lib import get_roadmap_issues

In [7]:
jira_roadmap_issues = get_roadmap_issues(
    jira_service=jira_client, 
    jira_project='CDFNEW',
    issue_type='Roadmap Initiative',
    product_category_mode = 'components',
    product_category_prefix = 'Product - ',
    include_beta=True,
    beta_attribute_name='customfield_10933'
)

AttributeError: 'NoneType' object has no attribute 'value'

In [8]:
jira_roadmap_issues[0]

NameError: name 'jira_roadmap_issues' is not defined

In [9]:
roadmap_issue_ids = jira_client.search_issues(
    jql_str='project = CDFNEW and issuetype = "Roadmap Initiative" and status in ("Q1", "Q2", "Q3", "Q4") ORDER BY Rank ASC',
    maxResults=None
)

In [16]:
issue = jira_client.issue(id=roadmap_issue_ids[4])

In [17]:
issue.fields.summary

'Leverage Generative AI co-pilot\n'

In [18]:
getattr(issue.fields, "customfield_10933")

In [55]:
issue.fields.summary

'AI generated apps/dashboards using Streamlit'

In [59]:
issue.fields.assignee

<JIRA User: displayName='Lars Moastuen', accountId='5ce26e623e6fd40fe82cb171'>

In [85]:
issue.fields.customfield_10933

<JIRA CustomFieldOption: value='Beta', id='11961'>

In [82]:
for att in dir(issue.fields):

    print(att, getattr(issue.fields, att))
    
    

__class__ <class 'jira.resources.PropertyHolder'>
__delattr__ <method-wrapper '__delattr__' of PropertyHolder object at 0x108ff29a0>
__dict__ {'customfield_10870': None, 'customfield_10750': None, 'resolution': None, 'customfield_10871': None, 'customfield_10630': None, 'customfield_10872': None, 'customfield_10752': None, 'customfield_10631': None, 'customfield_10510': None, 'customfield_10500': None, 'customfield_10742': None, 'customfield_10863': None, 'customfield_10864': None, 'customfield_10501': None, 'customfield_10743': None, 'customfield_10502': None, 'customfield_10623': None, 'customfield_10744': None, 'customfield_10865': None, 'customfield_10503': None, 'customfield_10745': None, 'customfield_10624': None, 'customfield_10625': None, 'customfield_10504': None, 'customfield_10746': None, 'customfield_10747': None, 'customfield_10626': None, 'customfield_10505': None, 'customfield_10506': None, 'customfield_10748': None, 'customfield_10627': None, 'customfield_10869': None, 

## Combining Everything

### Using Components or Labels to Generate Sections

In [13]:
def get_unique_product_groups(roadmap_issues):

    categories = []
    
    for issue in roadmap_issues:

        categories += issue.product_categories

    return list(set(categories))

@dataclass
class RoadmapSlide:
    title:str
    google_slide_id:str
    product_category:str

def generate_roadmap_slides(presentation_id, slides_service, product_categories, roadmap_slide_config):
    '''Creates the section headers and placeholder roadmap slides to populate with roadmap items

    Args:
        presentation_id (str): google id of presentation to add slides to
        slides_service (): google authenticated service to use when creating slides
        product_categories (list(str)): list of unique product categories
        roadmap_slide_config (dict): all of the properties required to create the roadmap slides

    Returns:
        (list(RoadmapSlides)): slides generated 
    '''
    
    slide_reqs = []
    slides = []
    
    for category in product_categories:
    
        header_slide_req, header_slide_id = gen_header_slide_req(title=category)
    
        slide_reqs += header_slide_req    
    
        roadmap_slide_req, roadmap_slide_id = gen_roadmap_slide_req(
            title = category, 
            roadmap_slide_config = roadmap_slide_config
        )
    
        slides.append(RoadmapSlide(
            title=category,
            google_slide_id=roadmap_slide_id,
            product_category=category
        ))
        
        slide_reqs += roadmap_slide_req

    res = updateSlides(
        service = slides_service,
        presentationId = presentation_id,
        body = {'requests': slide_reqs}
    )

    return slides

In [10]:
product_categories = get_unique_product_groups(jira_roadmap_issues)
slides = generate_roadmap_slides(
    slides_service=slides_service, 
    presentation_id='1XDRX_Bq39GTT8_6r2oorklhkw-MuIpn-VqFUSHn_Fdo', 
    product_categories=product_categories,
    roadmap_slide_config=roadmap_slide_config
)

NameError: name 'roadmap_slide_config' is not defined

### Iterate through Jira Issues to create shapes

In [9]:
def populate_roadmap_with_issues(
    presentation_id, 
    slides_service, 
    roadmap_slides, 
    curr_quarter, 
    roadmap_slide_config
):
    '''Places the roadmap items on the slides provided based on the config

    Args:
        presentation_id (str): google id of presentation to add slides to
        slides_service (): google authenticated service to use when creating slides
        roadmap_slides (list(Slide)): slides for placing the roadmap items on
        curr_quarter (int): current quarter (1 - 4)
        roadmap_slide_config (dict): configuraiton for the roadmap slides

    Returns: 
        (list): updates made to the slides
    '''

    roadmap_box_config = roadmap_slide_config['roadmap_box']
    arrow_config = roadmap_slide_config['timeline_arrow']
    left_header_config = roadmap_slide_config['left_header']
    quarters_config = roadmap_slide_config['quarters']
    quarter_marker_config = roadmap_slide_config['quarter_marker']
    columns_config = roadmap_slide_config['columns']

    roadmap_box_width = (arrow_config['width'] - roadmap_box_config['x_padding'] * (quarters_config['num_quarter_groups'] + 1)) \
        / quarters_config['num_quarter_groups']
    roadmap_box_locx = left_header_config['locx'] + left_header_config['width'] + roadmap_box_config['x_padding']
    roadmap_box_locy = left_header_config['locy'] + left_header_config['height']
    
    roadmap_shapes = []

    for slide in roadmap_slides:

        for col in columns_config:
            col['count'] = 0
    
        for issue in jira_roadmap_issues:
    
            if slide.product_category in issue.product_categories:

                for col_num, col in enumerate(columns_config):
                    
                    if issue.jira_quarter in col['jira_statuses']:

                        locx = roadmap_box_locx + (roadmap_box_width + roadmap_box_config["x_padding"]) * col_num

                        locy = col['count'] * (roadmap_box_config["height"] + roadmap_box_config["y_padding"]) + roadmap_box_locy

                        col['count'] += 1

                        roadmap_shape, shape_id = gen_roadmap_item_req(
                            page_id=slide.google_slide_id,
                            tagline=issue.summary,
                            description=issue.description[:roadmap_box_config["description_length"]],
                            width=roadmap_box_width,
                            locx=locx,
                            locy=locy,
                            link=issue.jira_link,
                            roadmap_box_config=roadmap_box_config
                        )
                
                        roadmap_shapes.append(roadmap_shape)
    
    return updateSlides(
        service = slides_service,
        presentationId = presentation_id,
        body = {'requests': roadmap_shapes}
    )

## Final structure

### Config file

In [2]:
roadmap_slide_config = {
    "jira_roadmap_issues":{
        "jql_filter":'project = CDFNEW and issuetype = "Roadmap Initiative" and status in ("Q1", "Q2", "Q3", "Q4") ORDER BY Rank ASC',
        "product_category_mode": 'components',
        "product_category_prefix":'Product - '
    },
    "left_header": {
        "shape_type":'ROUND_RECTANGLE',
        "locx":35,
        "locy":100,
        "height":30,
        "width":170,
        "color":'ACCENT1',
        "font_size":12
    },
    "left_main":{
        "shape_type":'RECTANGLE',
        "locy":100,
        "height":280,
        "width":170,
        "color":'LIGHT2',
        "font_size":10,
        "text":"Describe the product category in a few sentences with 1 - 3 bulleted themes for the roadmap."}
    ,
    "timeline_arrow":{
        "color":'DARK1',
        "weight":2,
        "width":480
    },
    "quarter_marker":{
        "shape_type": "DIAMOND",
        "height":20, 
        "width":20, 
        "color":"ACCENT1",
        "font_size":14,
        "font":"Manrope",
        "font_color":"DARK1"
    },
    "roadmap_box": {
        "description_length":200,
        "height":50,
        "font_size":7,
        "font_color":"DARK1",
        "x_padding":5,
        "y_padding":5,
        "fill_color":"LIGHT1",
        "outline_color":"ACCENT1",
        "shape_type":"ROUND_RECTANGLE"
    },
    "quarters":{
        "quarter_jira_statuses":{
            1: "Q1",
            2: "Q2",
            3: "Q3",
            4: "Q4"
        },
        "num_quarter_groups": 3,
        "group_quarters": [3,4]
    },
    "columns": [
        {
            "jira_statuses":["Q2"],
            "label":"Q2 2024"
        },
        {
            "jira_statuses":["Q3"],
            "label":"Q3 2024"
        },
        {
            "jira_statuses":["Q4","Q1"],
            "label":"Q4 2024 - Q1 2025"
        }
    ]
}

In [40]:
with open('config.json', 'w') as fp:
    json.dump(roadmap_slide_config, fp)

### Authenticate

In [2]:
jira_roadmap_issues = get_roadmap_issues(
    jira_service=jira_client, 
    **roadmap_slide_config['jira_roadmap_issues']
)

product_categories = get_unique_product_groups(jira_roadmap_issues)

slides = generate_roadmap_slides(
    slides_service=google_client, 
    presentation_id=PRESENTATION_ID, 
    product_categories=product_categories,
    roadmap_slide_config=roadmap_slide_config
)

res_populate_roadmap = populate_roadmap_with_issues(
    presentation_id=PRESENTATION_ID,
    slides_service=google_client,
    roadmap_slides=slides, 
    roadmap_slide_config=roadmap_slide_config,
    jira_roadmap_issues=jira_roadmap_issues
)

In [6]:
def generate_roadmap_deck(jira_service, google_service, roadmap_slide_config, presentation_id):
    '''Get the jira roadmap issues and generate all of the slides with the details
    
    Args: 
        jira_service (JIRA): authenticated service used for interacting with jira
        google_service (): google authenticated service to use when creating slides
        roadmap_slide_config (dict): configuraiton for the roadmap slides 
        presentation_id (str): google id of presentation to add slides to
    '''

    jira_roadmap_issues = get_roadmap_issues(
        jira_service=jira_service, 
        **roadmap_slide_config['jira_roadmap_issues']
    )
    
    product_categories = get_unique_product_groups(jira_roadmap_issues)
    
    slides = generate_roadmap_slides(
        slides_service=google_service, 
        presentation_id=presentation_id, 
        product_categories=product_categories,
        roadmap_slide_config=roadmap_slide_config
    )
    
    res_populate_roadmap = populate_roadmap_with_issues(
        presentation_id=presentation_id,
        slides_service=google_service,
        roadmap_slides=slides, 
        roadmap_slide_config=roadmap_slide_config,
        jira_roadmap_issues=jira_roadmap_issues
    )

    return f"Generated {len(slides)*2} slides, {len(product_categories)} product categories, and {len(jira_roadmap_issues)} roadmap items."

# Final

In [1]:
from lib import google_slides_connect, JIRA, generate_roadmap_deck
from getpass import getpass
import json

with open('config.json', 'r') as f:
    roadmap_slide_config = json.load(f)

google_client = google_slides_connect()
jira_client = JIRA(basic_auth=("hunter.beck@cognite.com", getpass('Jira API Key:')), options={'server': 'https://cognitedata.atlassian.net/'})

PRESENTATION_ID = '1XDRX_Bq39GTT8_6r2oorklhkw-MuIpn-VqFUSHn_Fdo'

/Users/hunterbeck/Repos/jira-slide-generator/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1076520620839-3cadpb9secdetc938k51obvoc2tl79sc.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpresentations&state=V4EuDnV9wTn0CZxYEktU20MOBz2vvK&access_type=offline


Jira API Key: ········


In [4]:
with open('config.json', 'r') as f:
    roadmap_slide_config = json.load(f)

generate_roadmap_deck(jira_client, google_client, roadmap_slide_config, PRESENTATION_ID)

'Generated 42 slides, 21 product categories, and 358 roadmap items.'

In [3]:
infield_items = get_roadmap_issues(
    jira_service=jira_cloud, 
    jql_filter='project = CDFNEW and issuetype = "Roadmap Initiative" and status in ("Q1", "Q2", "Q3", "Q4") ORDER BY Rank ASC',
    product_category_mode = 'components',
    product_category_prefix='Product - InField'
)

NameError: name 'get_roadmap_issues' is not defined

In [259]:
[item.description[:200] for item in infield_items]

['Set up alerts and notifications that trigger when thresholds or anomalies are detected for critical equipment and processes.',
 'Laying the grounds for a cohesive experience through the Cognite Data Fusion portfolio targeted towards workflows and user groups within designated workspaces. Supporting the quick start package, with',
 '*Single Sign-on*\xa0\n\nStandardized log-in across all Cognite products, enabled by Cognite IdP integrating with customer IdP',
 'In product help center - guided experience to complete a set of tutorials to become a novice user',
 'Improve the Asset Health offering by leveraging gen AI on data analysis while working in a collaborative canvas.\nCo-pilot in Industrial Canvas. \nContinuously adding AI features into canvas. \n\n[https:/',
 'Low-code client built applications in Python using Streamlit from Fusion',
 'Support for over 100 source system\xa0 relevant for Heavy asset Industry',
 'Extractor for the industrial process historian Aspentech InfoPlus. 21

In [265]:
d = {1:'one',2:'two', 3:'three'}

In [273]:
list(d.keys())[-1]

3

In [291]:
import datetime as datetime

In [294]:
datetime.datetime.now().year

2024

In [11]:
roadmap_slide_config['columns']

[{'jira_statuses': ['Q2'], 'label': 'Q2 2024'},
 {'jira_statuses': ['Q3'], 'label': 'Q3 2024'},
 {'jira_statuses': ['Q4', 'Q1'], 'label': 'Q4 2024 - Q1 2025'}]

In [13]:
for col in roadmap_slide_config['columns']:
    col['count'] = 1